In [6]:
from pyssem.pyssem import Model
# from pyssem.pyssem.utils.launch.launch import assign_species_to_population
import json
import numpy
import pickle
import os
import pandas as pd


In [7]:
def assign_species_to_population(T, species_mapping):
    """
    Applies a list of pandas query strings to assign species classes to the population.
    
    :param T: pandas.DataFrame representing the population
    :param species_mapping: list of assignment strings (e.g., T.loc[...] = ...)
    :return: updated DataFrame with 'species_class' assigned
    """
    # Initialize the column
    T['species_class'] = "Unknown"

    # Apply each mapping rule via exec
    for rule in species_mapping:
        try:
            exec(rule)
        except Exception as e:
            print(f"Error applying rule: {rule}\n\t{e}")

    # Print summary of resulting species_class assignments
    print("\nSpecies class distribution:")
    print(T['species_class'].value_counts())

    try:
        print(f"\n{T['species_class'].value_counts()['Unknown']} objects/rows are being removed.")
        T = T[T['species_class'] != "Unknown"]
    except KeyError:
        print("No unknown species classes found.")

    return T

In [18]:
# Load SEP

file_path = "pyssem/utils/launch/data/ref_scen_SEP3H.csv"
sep = pd.read_csv(file_path)

# Rule Set
SEP_mapping = [
      "T.loc[(T['phase'] == 2) & (T['obj_type'] == 2) & (T['maneuverable'] == 1), 'species_class'] = 'S'",
      "T.loc[(T['obj_type'] == 2) & (T['maneuverable'] == 0), 'species_class'] = 'Sns'",
      "T.loc[(T['obj_type'] >= 3), 'species_class'] = 'N'",
      "T.loc[(T['obj_type'] == 1), 'species_class'] = 'B'"
    ]

test = assign_species_to_population(sep, SEP_mapping)


Species class distribution:
species_class
Sns        2737445
S          2317088
Unknown     426392
N            54731
B            19475
Name: count, dtype: int64

426392 objects/rows are being removed.


In [17]:
# print(test['species_class'][1])
print(test['species_class'].unique())

print(test['species_class'].value_counts())

# print(test['species_class'].value_counts()['Unknown'])

['Sns' 'B' 'N' 'S']
species_class
S      344686
Sns    341136
N       54722
B       13212
Name: count, dtype: int64
